# RTpipeline on Google Colab - Part 1: GPU Segmentation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/rtpipeline/blob/main/rtpipeline_colab_part1_gpu.ipynb)

**💰 Cost Optimization:** This notebook is split into two parts to optimize GPU costs:
- **Part 1 (this notebook):** Runs TotalSegmentator with GPU (~10-30 min/patient)
- **Part 2:** Runs DVH, radiomics, and analysis on CPU only (saves GPU costs)

## What This Part Does

✅ **Automatic segmentation** of 100+ organs using TotalSegmentator (GPU-accelerated)
✅ **Saves outputs** to Google Drive for Part 2

## Prerequisites

- Google Colab with **GPU runtime** (Runtime → Change runtime type → GPU)
- DICOM files in Google Drive
- Google Drive mounted for saving outputs

---

**⚡ Quick Start:** 
1. Run cells 1-3 (setup)
2. Mount Google Drive (cell 4)
3. **UPDATE CONFIGURATION** (cell 5) - Point to your DICOM folder
4. Run remaining cells

## 1️⃣ Setup: Install Miniconda & System Dependencies

This takes ~2 minutes

In [ ]:
%%bash
# Check GPU availability
echo "=== GPU Check ==="
nvidia-smi || echo "⚠️ No GPU detected. Please enable GPU: Runtime → Change runtime type → GPU"

# Install system dependencies
echo -e "\n=== Installing System Dependencies ==="
apt-get update -qq
apt-get install -y -qq dcm2niix pigz > /dev/null

# Install Miniconda if not already installed
if [ ! -d "/content/miniconda" ]; then
    echo -e "\n=== Installing Miniconda ==="
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    bash /tmp/miniconda.sh -b -p /content/miniconda
    rm /tmp/miniconda.sh
    echo "✅ Miniconda installed"
else
    echo "✅ Miniconda already installed"
fi

# Initialize conda
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"
conda init bash

echo -e "\n✅ Setup complete!"

## 2️⃣ Clone RTpipeline Repository

In [ ]:
%%bash
if [ ! -d "/content/rtpipeline" ]; then
    echo "Cloning rtpipeline repository..."
    git clone -q https://github.com/kstawiski/rtpipeline.git /content/rtpipeline
    echo "✅ Repository cloned"
else
    echo "✅ Repository already exists"
    cd /content/rtpipeline
    git pull origin main
    echo "Repository updated"
fi

## 3️⃣ Create Conda Environment

This creates the rtpipeline environment for TotalSegmentator (~5-10 minutes, only once per session)

In [ ]:
%%bash
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"

# Accept Anaconda Terms of Service
echo "=== Accepting Anaconda Terms of Service ==="
conda config --set channel_priority flexible
if ! conda tos accept --channel defaults 2>&1; then
    echo "⚠️ ToS acceptance failed or already accepted"
fi
echo "✅ ToS accepted"

cd /content/rtpipeline

# Create rtpipeline environment
if conda env list | grep -q "^rtpipeline "; then
    echo "✅ Environment 'rtpipeline' already exists"
else
    echo "Creating 'rtpipeline' environment (TotalSegmentator)..."
    conda env create -f envs/rtpipeline.yaml -q
    echo "✅ Environment created"
fi

echo ""
conda run -n rtpipeline python -c "import numpy; print(f'✅ numpy {numpy.__version__}')"

## 4️⃣ Mount Google Drive

**IMPORTANT:** Your DICOM files must be in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted at /content/drive/MyDrive/")

---

# ⚙️ CONFIGURATION - UPDATE THIS!

## 5️⃣ Configure Input/Output Paths

**🔴 REQUIRED:** Update `DICOM_ROOT` to point to your DICOM files in Google Drive

---

In [ ]:
import os
import subprocess

# ═══════════════════════════════════════════════════════════
# 🔴 UPDATE THIS - Point to your DICOM folder in Google Drive
# ═══════════════════════════════════════════════════════════

DICOM_ROOT = "/content/drive/MyDrive/my_dicom_folder"

# Examples:
# DICOM_ROOT = "/content/drive/MyDrive/RT_Data/DICOM"
# DICOM_ROOT = "/content/drive/MyDrive/Patient_Data"

# ═══════════════════════════════════════════════════════════
# Output location in Google Drive (for Part 2)
# ═══════════════════════════════════════════════════════════

DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/rtpipeline_part1_output"

# ═══════════════════════════════════════════════════════════
# Optional: Processing settings (usually no need to change)
# ═══════════════════════════════════════════════════════════

WORKERS = 2              # I/O parallelism (2-4 recommended)
SEG_WORKERS = 1          # GPU workers (keep at 1 for single GPU)
FAST_MODE = False        # True = faster but lower quality
ENABLE_CUSTOM_MODELS = False  # Enable custom nnUNet models

# ═══════════════════════════════════════════════════════════
# Validation
# ═══════════════════════════════════════════════════════════

# Local working directories
OUTPUT_DIR = "/content/output"
LOGS_DIR = "/content/logs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

# Check GPU
try:
    subprocess.run(['nvidia-smi'], check=True, capture_output=True)
    gpu_available = True
    print("✅ GPU detected")
except:
    gpu_available = False
    print("⚠️ No GPU detected!")
    print("   Please enable GPU: Runtime → Change runtime type → GPU")

# Check DICOM directory
if not os.path.exists(DICOM_ROOT):
    print(f"\n🔴 ERROR: DICOM directory not found!")
    print(f"   Path: {DICOM_ROOT}")
    print(f"\n   Please update DICOM_ROOT in the cell above.")
    print(f"   Your DICOM files must be in Google Drive.")
else:
    dicom_count = sum(1 for root, dirs, files in os.walk(DICOM_ROOT) 
                      for f in files if f.endswith('.dcm'))
    print(f"\n✅ DICOM directory found: {DICOM_ROOT}")
    print(f"   {dicom_count} DICOM files detected")

print(f"\n📋 Configuration Summary:")
print(f"   GPU: {'✅ Available' if gpu_available else '❌ Not available'}")
print(f"   Workers: {WORKERS}")
print(f"   Fast Mode: {'✅ Enabled' if FAST_MODE else '❌ Disabled'}")
print(f"\n📦 Outputs will be saved to: {DRIVE_OUTPUT_DIR}_YYYYMMDD_HHMMSS")

---

## 6️⃣ Generate Configuration File

This creates the pipeline configuration based on your settings above.

In [ ]:
config_yaml = f"""# RTpipeline Configuration - Part 1 (GPU Segmentation)
dicom_root: "{DICOM_ROOT}"
output_dir: "{OUTPUT_DIR}"
logs_dir: "{LOGS_DIR}"
workers: {WORKERS}

segmentation:
  workers: {SEG_WORKERS}
  fast: {str(FAST_MODE).lower()}
  device: "{'gpu' if gpu_available else 'cpu'}"
  force_split: true
  nr_threads_resample: 1
  nr_threads_save: 1
  num_proc_preprocessing: 1
  num_proc_export: 1

custom_models:
  enabled: {str(ENABLE_CUSTOM_MODELS).lower()}

custom_structures: "custom_structures_pelvic.yaml"
"""

config_path = "/content/config_part1.yaml"
with open(config_path, 'w') as f:
    f.write(config_yaml)

print(f"✅ Configuration written to: {config_path}")

## 7️⃣ Run Segmentation Pipeline

This runs **ONLY** TotalSegmentator segmentation (GPU-accelerated)

⏱️ **Time:** 10-30 minutes per patient with GPU

In [ ]:
import os
import subprocess
import glob

os.environ['PATH'] = f"/content/miniconda/bin:{os.environ.get('PATH', '')}"
os.chdir('/content/rtpipeline')

print("═══════════════════════════════════════════════════")
print("   RTpipeline Part 1: GPU Segmentation")
print("═══════════════════════════════════════════════════")
print("\nThis will run ONLY segmentation (GPU-accelerated)")
print("DVH and radiomics will run in Part 2 (CPU)\n")

# Install Snakemake if needed
try:
    subprocess.run(["conda", "run", "-n", "base", "snakemake", "--version"],
                   check=True, capture_output=True)
except subprocess.CalledProcessError:
    print("Installing Snakemake...")
    subprocess.run(["conda", "install", "-n", "base", "-c", "conda-forge", 
                    "-c", "bioconda", "snakemake", "-y", "-q"], check=True)
    print("✅ Snakemake installed\n")

# Step 1: Organize courses
print("[1/2] Organizing DICOM data...")
cmd_organize = [
    "conda", "run", "-n", "base", "snakemake",
    "--configfile", "/content/config_part1.yaml",
    "--use-conda", "--cores", str(WORKERS),
    "--printshellcmds",
    "/content/output/_COURSES/manifest.json"
]

result = subprocess.run(cmd_organize, capture_output=False, text=True)

if result.returncode != 0:
    print("\n⚠️ Organization failed!")
else:
    print("\n✅ Organization complete\n")
    
    # Step 2: Run segmentation
    print("[2/2] Running TotalSegmentator...")
    
    # Find all courses
    seg_targets = []
    custom_targets = []
    
    for patient_dir in glob.glob(f"{OUTPUT_DIR}/*/"):
        patient_name = os.path.basename(patient_dir.rstrip('/'))
        if patient_name.startswith('_') or patient_name.startswith('.'):
            continue
        for course_dir in glob.glob(f"{patient_dir}/*/"):
            course_name = os.path.basename(course_dir.rstrip('/'))
            if not course_name.startswith('_'):
                seg_targets.append(f"{OUTPUT_DIR}/{patient_name}/{course_name}/.segmentation_done")
                custom_targets.append(f"{OUTPUT_DIR}/{patient_name}/{course_name}/.custom_models_done")
    
    if seg_targets:
        print(f"Found {len(seg_targets)} course(s) to segment\n")
        
        cmd_seg = [
            "conda", "run", "-n", "base", "snakemake",
            "--configfile", "/content/config_part1.yaml",
            "--use-conda", "--cores", str(WORKERS),
            "--printshellcmds", "--keep-going"
        ] + seg_targets + custom_targets
        
        result = subprocess.run(cmd_seg, capture_output=False, text=True)
        
        if result.returncode == 0:
            print("\n✅ All segmentations complete!")
        else:
            print("\n⚠️ Some segmentations failed. Check logs.")
    else:
        print("\n⚠️ No courses found")

print("\n" + "="*50)
print("Part 1 Complete!")
print("="*50)
print(f"\nOutputs: {OUTPUT_DIR}")
print("\nNext: Run the cell below to save to Google Drive")

## 8️⃣ Save Outputs to Google Drive

**IMPORTANT:** This saves your segmentation results to Google Drive for Part 2

In [ ]:
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
drive_output = f"{DRIVE_OUTPUT_DIR}_{timestamp}"

print(f"Copying outputs to Google Drive...")
print(f"Destination: {drive_output}\n")

try:
    shutil.copytree(OUTPUT_DIR, drive_output)
    shutil.copy("/content/config_part1.yaml", f"{drive_output}/config_part1.yaml")
    
    # Create README for Part 2
    readme = f"""RTpipeline Part 1 Outputs
========================
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

To continue with Part 2:
1. Open rtpipeline_colab_part2_cpu.ipynb
2. In cell 5 (Configuration), set:
   PART1_OUTPUT_DIR = "{drive_output}"
3. Run all cells (on CPU runtime - no GPU needed!)

DICOM source: {DICOM_ROOT}
"""
    with open(f"{drive_output}/README_PART2.txt", 'w') as f:
        f.write(readme)
    
    print("\n" + "="*60)
    print("🎉 PART 1 COMPLETE - OUTPUTS SAVED!")
    print("="*60)
    print(f"\nSaved to: {drive_output}")
    print("\n📋 Next Steps:")
    print("   1. You can disconnect this GPU runtime now")
    print("   2. Open rtpipeline_colab_part2_cpu.ipynb")
    print(f"   3. Set PART1_OUTPUT_DIR = '{drive_output}'")
    print("   4. Run Part 2 on CPU runtime (saves GPU costs!)")
    
except Exception as e:
    print(f"\n⚠️ Error: {e}")
    print("\nPlease check:")
    print("  - Google Drive is mounted")
    print("  - You have enough space")
    print("  - Path is valid")

## 📊 Optional: View Summary

In [ ]:
import os
import glob

print("═══════════════════════════════════")
print("   Segmentation Summary")
print("═══════════════════════════════════\n")

total = 0
completed = 0

for patient_dir in sorted(glob.glob(f"{OUTPUT_DIR}/*/")):
    patient_name = os.path.basename(patient_dir.rstrip('/'))
    if patient_name.startswith('_') or patient_name.startswith('.'):
        continue
    
    for course_dir in sorted(glob.glob(f"{patient_dir}/*/")):
        course_name = os.path.basename(course_dir.rstrip('/'))
        if course_name.startswith('_'):
            continue
        
        total += 1
        seg_done = os.path.exists(f"{course_dir}/.segmentation_done")
        
        if seg_done:
            completed += 1
        
        status = "✅" if seg_done else "⚠️"
        print(f"{status} {patient_name}/{course_name}")

print(f"\nTotal: {completed}/{total} completed")

if completed == total and total > 0:
    print("\n🎉 All segmentations successful!")
elif completed > 0:
    print(f"\n⚠️ {total - completed} incomplete")
else:
    print("\n⚠️ No segmentations completed")

---

## What's Next?

**Continue with Part 2 (CPU):** `rtpipeline_colab_part2_cpu.ipynb`

Part 2 will:
- Extract DVH metrics
- Compute radiomic features
- Run robustness testing (optional)
- Generate visualizations
- Create downloadable results

**💰 Cost Savings:** Part 2 runs on CPU only!

---

**Notebook Version:** 2.0 (Part 1 - GPU Segmentation)  
**Repository:** https://github.com/kstawiski/rtpipeline